In [10]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.nn as nn
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np
import utils

In [11]:
# Define the mini-batch size
batch_size = 1000

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224)
])

# Download the dataset and create the dataloaders
cifar_data = datasets.CIFAR10("./", train=True, download=True, transform=transform)

train_size = int(0.8 * len(cifar_data))
val_size = len(cifar_data)-train_size

mnist_train, mnist_val = torch.utils.data.random_split(cifar_data, [train_size, val_size])

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(mnist_val, batch_size=batch_size, shuffle=False)

mnist_test = datasets.MNIST("./", train=False, download=True, transform=transform)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [12]:
# fine tuning
model_ft = models.alexnet(pretrained=True)

# Do the things required for fine-tuning before training the model
# modifying last layer to our need
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 10)

LEARNING_RATE = 0.0001
EPOCHS = 20

# Define our loss function
criterion_ft = nn.CrossEntropyLoss()

# Define our optimizer
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)

# Train the model
utils.train_model(model_ft, criterion_ft, optimizer_ft, train_loader, val_loader,"best_model_fine_tuning.pt", EPOCHS)

c:\Users\britta\anaconda3\envs\nnlm\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\britta\anaconda3\envs\nnlm\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\britta\anaconda3\envs\nnlm\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning,

KeyboardInterrupt: 

In [ ]:
# Test the model
best_model = torch.load('best_model_fine_tuning.pt')

utils.test_model(best_model, test_loader)

In [ ]:
# Use a predefined model as a feature extractor
model_fe = models.alexnet(pretrained=True)

for param in model_fe.parameters():
    param.requires_grad = False

# Do the things required for fine-tuning before training the model
LEARNING_RATE = 0.0001
EPOCHS = 20

num_ftrs = model_fe.classifier[6].in_features
model_fe.fc = nn.Linear(num_ftrs, 2)

# Unfreeze last layer
for param in model_fe.classifier[6].parameters():
    param.requires_grad = True

# Define our loss function
criterion_fe = nn.CrossEntropyLoss()

# Define our optimizer
optimizer_fe = torch.optim.Adam(model_fe.parameters(), lr=LEARNING_RATE)

# Train the model
utils.train_model(model_fe, criterion_fe, optimizer_fe, train_loader, val_loader,'best_model_feature_extraction.pt', EPOCHS)

RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small

In [ ]:
# Test the model
best_model = torch.load('best_model_feature_extraction.pt')

utils.test_model(best_model, test_loader)